<a href="https://colab.research.google.com/github/Hubert26/suicides_IPPAN/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import os
from google.colab import files

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None

In [2]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/final_samobojstwa_2013_2022.csv', delimiter=',', low_memory=False)

In [3]:
df_raw.head()

,ID samobójcy,Data raportu [RRRRMM],Przedział wiekowy,Płeć,Stan cywilny,Wykształcenie,Informacje o pracy i nauce,Źródło utrzymania,Czy samobójstwo zakończyło się zgonem,Miejsce zamachu,Sposób popełnienia,Powód zamachu,Powód zamachu 2,Powód zamachu 3,Powód zamachu 4,Stan świadomości,Informacje o używaniu substancji,Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii
0,"114581552,00",201301,0,2,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0
1,"114584284,00",201301,2,2,0,0,1,0,1,1,1,0,NaN,NaN,NaN,0,0,0
2,"114584293,00",201301,2,1,1,2,1,1,1,15,2,0,NaN,NaN,NaN,0,0,0
3,"114584312,00",201301,3,2,1,0,0,0,1,0,18,1,NaN,NaN,NaN,0,0,0
4,"114584399,00",201301,7,2,1,0,9,0,1,2,3,0,NaN,NaN,NaN,2,0,0


#Zamiana nazw kolumn

In [4]:
# Zamiana spacji na podkreślenia w nazwach kolumn
df_raw.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)


In [5]:
df_raw.rename(columns={'Data_raportu_[RRRRMM]': 'Data'}, inplace=True)

# Daty

In [6]:
# Wyodrębnienie roku i miesiąca
df_raw['Data_rok'] = df_raw['Data'].apply(lambda x: x[0:4] if len(x) > 8 else x[:4])
df_raw['Data_miesiąc'] = df_raw['Data'].apply(lambda x: x[5:7] if len(x) > 8 else x[4:6])

In [7]:
# Połączenie roku i miesiąca w formacie MM.YYYY
df_raw['Data'] = df_raw['Data_miesiąc'] + '.' + df_raw['Data_rok']

#Mapowanie

##Przedział_wiekowy

In [8]:
column = 'Przedział_wiekowy'
mapping = {
    0: np.nan,
    1: '07-12',
    2: '13-18',
    3: '19-24',
    4: '25-29',
    5: '30-34',
    6: '35-39',
    7: '40-44',
    8: '45-49',
    9: '50-54',
    10: '55-59',
    11: '60-64',
    12: '65-69',
    13: '70-74',
    14: '75-79',
    15: '80-84',
    16: '85+'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Wiek'}, inplace=True)

##Płeć

In [9]:
column = 'Płeć'
mapping = {
    0: np.nan,
    1: 'K',
    2: 'M'
}

df_raw[column] = df_raw[column].map(mapping)


##Stan_cywilny

In [10]:
column = 'Stan_cywilny'
mapping = {
    0: np.nan,
    1: 'Samotność',
    2: 'Partnerstwo',
    3: 'Małżeństwo',
    4: 'Separacja',
    5: 'Rozwód',
    6: 'Wdowieństwo'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'St_cyw'}, inplace=True)


##Wykształcenie

In [11]:
column = 'Wykształcenie'
mapping = {
    0: np.nan,
    1: 'Podst. niepełne',
    2: 'Podstawowe',
    3: 'Gimnazjalne',
    4: 'Zasadnicze zaw.',
    5: 'Średnie',
    6: 'Policealne',
    7: 'Wyższe'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Wyksz.'}, inplace=True)



##Informacje_o_pracy_i_nauce

In [12]:
column = 'Informacje_o_pracy_i_nauce'
mapping = {
    0: np.nan,
    1: 'Uczeń',
    2: 'Student',
    3: 'Umysłowa',
    4: 'Fizyczna',
    5: 'Rolnik',
    6: 'Własna',
    7: 'Stała',
    8: 'Dorywcza',
    9: 'Bezrobotny'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Inf_praca'}, inplace=True)

##Źródło_utrzymania

In [13]:
column = 'Źródło_utrzymania'
mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 'Zależny',
    2: 'Praca',
    3: 'Świadczenia',
    4: 'Bez dochodu'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Utrzymanie'}, inplace=True)

##Czy_samobójstwo_zakończyło_się_zgonem

In [14]:
column = 'Czy_samobójstwo_zakończyło_się_zgonem'

mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 1,
    2: 0
}
df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Czy_zgon'}, inplace=True)

##Miejsce_zamachu

In [15]:
column = 'Miejsce_zamachu'
mapping = {
    0: np.nan,
    1: 'Droga',
    2: 'Zabudowania',
    3: 'Mieszkanie',
    4: 'Tory',
    5: 'Park',
    6: 'Piwnica',
    7: 'Woda',
    8: 'Praca',
    9: 'Lecznica',
    10: 'Izolacja',
    11: 'Wojsko',
    12: 'Opieka',
    13: 'Szkoła',
    14: 'Policja',
    15: 'Inne'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Miejsce'}, inplace=True)


##Sposób_popełnienia

In [16]:
column = 'Sposób_popełnienia'
mapping = {
    0: np.nan,
    1: 'Pod pojazd',
    2: 'Z wysokości',
    3: 'Powieszenie',
    4: 'Układ krwionośny',
    5: 'Zastrzelenie',
    6: 'Samookaleczenie pow.',
    7: 'Samookaleczenie inne',
    8: 'Środki nasenne',
    9: 'Trucizna',
    10: 'Gaz',
    11: 'Inne leki',
    12: 'Środki chemiczne',
    13: 'Odurzające',
    14: 'Dopalacze',
    15: 'Utonięcie',
    16: 'Podpalenie',
    17: 'Uduszenie',
    18: 'Inny'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Sposób'}, inplace=True)

##Powód_zamachu

In [17]:
column = 'Powód_zamachu'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby psych',
    3: 'Konflikty rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby fiz',
    14: 'Zdrowie',
    15: 'Konflikty z innymi',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Powód'}, inplace=True)

##Powód_zamachu_2

In [18]:
column = 'Powód_zamachu_2'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby psych',
    3: 'Konflikty rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby fiz',
    14: 'Zdrowie',
    15: 'Konflikty z innymi',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Powód_2'}, inplace=True)

##Powód_zamachu_3

In [19]:
column = 'Powód_zamachu_3'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby psych',
    3: 'Konflikty rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby fiz',
    14: 'Zdrowie',
    15: 'Konflikty z innymi',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Powód_3'}, inplace=True)

##Powód_zamachu_4

In [20]:
column = 'Powód_zamachu_4'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby psych',
    3: 'Konflikty rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby fiz',
    14: 'Zdrowie',
    15: 'Konflikty z innymi',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Powód_4'}, inplace=True)

##Stan_świadomości

In [21]:
column = 'Stan_świadomości'
mapping = {
    0: np.nan,
    1: 'Trzeźwy',
    2: 'Alko',
    3: 'Dopa',
    4: 'Narko',
    5: 'Leki',
    6: 'Odurza',
    7: 'Alko+dopa',
    8: 'Alko+narko',
    9: 'Alko+leki',
    10: 'Alko+odurza',
    11: 'Alko+dopa+odurza',
    12: 'Dopa+odurza',
    13: 'Leki+odurza',
    14: 'Leki+dopa',
    15: 'Leki+narko',
    16: 'Leki+dopa+odurza',
    17: 'Alko+leki+odurza',
    18: 'Alko+leki+dopa+odurza'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Stan'}, inplace=True)

##Informacje_o_używaniu_substancji

In [22]:
column = 'Informacje_o_używaniu_substancji'
mapping = {
    0: np.nan,
    1: 'Alko',
    2: 'Narko',
    3: 'Dopa',
    4: 'Alko+dopa',
    5: 'Narko+dopa',
    6: 'Alko+narko',
    7: 'Alko+narko+dopa'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Używanie'}, inplace=True)


##Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii

In [23]:
column = 'Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii'
mapping = {
    0: np.nan,
    1: 'Alko',
    2: 'Narko',
    3: 'Alko+narko'
}

df_raw[column] = df_raw[column].map(mapping)
df_raw.rename(columns={column: 'Leczenie'}, inplace=True)

#Powód zamachu - dummies

In [24]:
columns_to_drop = ['Powód', 'Powód_2', 'Powód_3', 'Powód_4']
df_powód_zamachu = df_raw[columns_to_drop]
df_raw.drop(columns=columns_to_drop, inplace=True)


In [25]:
powód_zamachu_values = (
    set(df_powód_zamachu['Powód'].unique()) |
    set(df_powód_zamachu['Powód_2'].unique()) |
    set(df_powód_zamachu['Powód_3'].unique()) |
    set(df_powód_zamachu['Powód_4'].unique())
)

In [26]:
powód_zamachu_values

{'Bezdomność',
 'Choroba',
 'Choroby fiz',
 'Choroby psych',
 'Ciąża',
 'Finanse',
 'HIV/AIDS',
 'Inne',
 'Kalectwo',
 'Konflikty rodzinne',
 'Konflikty z innymi',
 'Miłość',
 'Mobbing',
 'Praca',
 'Przestępstwa',
 'Utrata utrzymania',
 'Zdrowie',
 nan,
 'Śmierć bliskiego'}

In [27]:
for value in powód_zamachu_values:
    column_name = 'Powód_' + str(value)
    df_powód_zamachu[column_name] = df_powód_zamachu.apply(lambda row: 1 if value in row.values else 0, axis=1)


<ipython-input-27-9a28d6c29fb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu[column_name] = df_powód_zamachu.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-27-9a28d6c29fb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu[column_name] = df_powód_zamachu.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-27-9a28d6c29fb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [28]:
columns_to_drop = ['Powód', 'Powód_2', 'Powód_3', 'Powód_4', 'Powód_nan']
df_powód_zamachu.drop(columns=columns_to_drop, inplace=True)


#Liczba_powodów_zamachu

In [29]:
df_raw['L_powodów'] = df_powód_zamachu.sum(axis=1)

In [30]:
df_raw.head()

,ID_samobójcy,Data,Wiek,Płeć,St_cyw,Wyksz.,Inf_praca,Utrzymanie,Czy_zgon,Miejsce,Sposób,Stan,Używanie,Leczenie,Data_rok,Data_miesiąc,L_powodów
0,"114581552,00",01.2013,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,01,0
1,"114584284,00",01.2013,13-18,M,NaN,NaN,Uczeń,NaN,1.0,Droga,Pod pojazd,NaN,NaN,NaN,2013,01,0
2,"114584293,00",01.2013,13-18,K,Samotność,Podstawowe,Uczeń,Zależny,1.0,Inne,Z wysokości,NaN,NaN,NaN,2013,01,0
3,"114584312,00",01.2013,19-24,M,Samotność,NaN,NaN,NaN,1.0,NaN,Inny,NaN,NaN,NaN,2013,01,1
4,"114584399,00",01.2013,40-44,M,Samotność,NaN,Bezrobotny,NaN,1.0,Zabudowania,Powieszenie,Alko,NaN,NaN,2013,01,0


#Zapis

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# df_raw.to_csv('/content/drive/My Drive/preped_samobojstwa_2013_2022.csv', index=False)
# df_powód_zamachu.to_csv('/content/drive/My Drive/preped_powod_zamachu_samobojstwa_2013_2022.csv', index=False)

In [31]:
file_name = 'preped_samobojstwa_2013_2022.csv'
df_raw.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
file_name = 'preped_powod_zamachu_samobojstwa_2013_2022.csv'
df_powód_zamachu.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>